import pandas 

In [8]:
import pandas as pd
import sentence_encoder

In [9]:
bing_data=pd.read_csv("../data/bing.csv",header=None)

In [10]:
import numpy as np
getty_data_pd=pd.read_csv("../data/cleaned_getty_data.csv",index_col=0)

getty_data_pd = getty_data_pd.replace(np.nan, "", regex=True)

In [11]:
getty_data_pd=getty_data_pd.reset_index()

In [12]:
big_encoder=sentence_encoder.SentenceTransform("roberta-base-nli-stsb-mean-tokens")

In [13]:
glove_encoder=sentence_encoder.SentenceTransform("average_word_embeddings_glove.6B.300d")

In [94]:
mean_caption_embedding=mega.view(6650,768).mean(axis=1)
mean_caption_embedding_numpy=[mean_caption_embedding.detach().numpy()]



In [165]:
#mean for full sentences 

def embed_mean(word_embedding_model,text_column:list,max_number:int):
    counter=0
    all_captions=[]
    for caption in text_column:
        if counter>max_number:
            break 
        if len(caption)==0:
            continue
        else:
            result=big_encoder.extract_text_features_simple([caption])
            all_captions.append(result)
        counter+=1

    mega=torch.Tensor(all_captions)
    return mega.view(mega.shape[0],mega.shape[2]).mean(dim=0)


def embed_column(word_embedding_model, text_column:list, mean_embedd, max_number:int):
    
    counter=0
    all_captions=[]
    for caption in getty_data_pd["caption"].values:
        if counter>max_number:
            break 
        if len(caption)==0:
            all_captions.append([mean_embedd])

        else:
            result=big_encoder.extract_text_features_simple([caption])
            all_captions.append(result)
        counter+=1
    return all_captions

In [ ]:
mean_embedding=embed_mean(big_encoder,getty_data_pd["caption"].values,50)
all_data=embed_column(big_encoder,getty_data_pd["caption"].values,mean_embedding,15)

In [211]:
getty_data_pd["best_tags"][0]

'Opioid   Syringe   Addiction   Despair   Heroin   Addict   Drug Overdose   Recreational Drug   Teenager   IV Drip   Abuse   Adult Amphetamine   Close-Up   Cocaine   Criminal   Danger   Dependency   Depression - Sadness   Emotional Stress   Forbidden   Frustration   Grief   Hand   Healthcare And Medicine   Horizontal   Human Body Part   Human Hand   Illness   Photography   Problems   Risk   Sharp   Snorting   Thailand   Vein   Worried   Photos'

In [222]:
result=glove_encoder.extract_text_features_simple(getty_data_pd["best_tags"][0].replace("  "," ").split(" "))

In [274]:

def embed_glove_tags(glove_embedding_model,tag_column_list,mean_embed, max_number):
    final_data=[]
    for i in range(max_number):
        tags_to_look_at=tag_column_list[i]
       
        
        result=_select_tags_glove(glove_embedding_model,tags_to_look_at)
  
        result=torch.Tensor(result).mean(dim=0).detach().numpy()
        final_data.append(result)
        
    return final_data
        
def _select_tags_glove(glove_embedding_model,tag_string:str):
    #just take out the tags that are actual words 
    tags=tag_string.replace("  "," ").split(" ")

    sub_tags=[]
    for tag in tags: 
        if(tag!=""):
            sub_tags.append(tag)
    
    
    temp_result=glove_embedding_model.extract_text_features_simple(sub_tags)
    final_final=[]

    for temp in temp_result:
        if temp[0]!=0:
  
            final_final.append(temp)
 
    return final_final



In [277]:
result=embed_glove_tags(glove_encoder,getty_data_pd["best_tags"].values,None,1000)

In [278]:
sentence_encoder.GettySentenceTransform()

torch.Size([1000, 300])

In [3]:
cls1=["Ethnicity","Religion","Sexual Orientation","Gender","Gender Identity","Disability Disease","Nationality",
      "Immigration Status"]
ex1=["African","Muslim","Lesbian","Male","Transgender","Autistic","Nationality","Migrant","Poor"]
ex2=["White","Christian","Gay","Woman","Queer","Cancer","Nationality","Undocumented","Rich"]
    
    

In [ ]:
glove_encoder

In [72]:
class ProtectedClassifierSimple():
    
    def __init__(self, sentence_encoder_model:sentence_encoder.SentenceTransform):
        cls1=["Ethnicity","Religion","Sexual Orientation","Gender","Gender Identity","Disability Disease","Nationality",
      "Immigration Status","socioeconomic Status"]
        ex1=["african","muslim","lesbian","male","transgender","autistic","nationality","migrant","poor"]
        ex2=["white","jewish","gay","woman","queer","cancer","nationality","undocumented","rich"]
        
        self.class_names=cls1
        
        self.transformer=sentence_encoder_model
        
        self.comparison_names=[]
        self.comparison_names.append(ex1)
        self.comparison_names.append(ex2)
        
        self.featurized_cl1=sentence_encoder_model.extract_text_features_simple(cls1)
        #self.featurized_ex1=sentence_encoder_model.extract_text_features_simple(ex1)
        #self.featurized_ex2=sentence_encoder_model.extract_text_features_simple(ex2)
        
    def measure_distance(self, sentences:list):
        
        transformed_sentences:list=self.transformer.extract_text_features_simple(sentences)
            
        #first= manhattan_distances(transformed_sentences,self.featurized_ex1)
        #second=manhattan_distances(transformed_sentences,self.featurized_ex2)
        third=manhattan_distances(transformed_sentences,self.featurized_cl1)
        
        return third #, np.var((first,second,third),axis=0)
        
        
        

In [68]:
t=ProtectedClassifierSimple(big_encoder)

In [69]:
from sklearn.metrics.pairwise import manhattan_distances

In [ ]:
t.measure_distance(["kill me","dog"])

In [ ]:
a=t.measure_distance(["nigger","jew","retard"])

In [71]:
a

array([[482.59958776, 498.66377375, 599.21235833, 549.47485269,
        564.35719589, 527.3622152 , 482.12608542, 563.26669934,
        535.74760391],
       [503.92217677, 496.59318416, 542.47142668, 544.35116678,
        561.4483124 , 532.68924958, 550.86909706, 552.19209644,
        535.16344658],
       [604.7842411 , 534.75539984, 610.30460624, 567.01158396,
        603.22137152, 459.59280359, 606.25180061, 570.24727427,
        543.93922696]])

In [65]:
b

array([[2.21925454e+03, 2.26203724e+03, 2.33195627e+03, 1.67984608e+02,
        1.28687521e+03, 1.28840382e+03, 1.15438061e-09, 2.35005113e+03,
        7.55686391e+02],
       [1.15023501e+02, 3.29921196e+04, 7.97214987e+02, 3.19175431e+02,
        6.11152723e+02, 9.85408460e+01, 9.35449067e-10, 3.44904628e+02,
        2.76897586e+02],
       [5.39704342e+02, 9.29523890e+02, 8.15851335e+02, 6.91416671e+02,
        2.29674230e+02, 4.01866856e+03, 1.39534305e-09, 3.16289666e+02,
        1.59720702e+03]])